In [ ]:
!pip install pytorch-adapt

### Setup

In [ ]:
import copy
from pprint import pprint

import torch

# Models
G = torch.nn.Linear(1000, 100)
C = torch.nn.Linear(100, 10)
D = torch.nn.Sequential(torch.nn.Linear(100, 1), torch.nn.Flatten(start_dim=0))
G_opt = torch.optim.Adam(G.parameters())
C_opt = torch.optim.Adam(C.parameters())
D_opt = torch.optim.Adam(D.parameters())

dataset_size = 10000
# 1 batch of data
example_data = {
    "src_imgs": torch.randn(32, 1000),
    "target_imgs": torch.randn(32, 1000),
    "src_labels": torch.randint(0, 10, size=(32,)),
    "src_domain": torch.zeros(32),
    "target_domain": torch.zeros(32),
    "src_sample_idx": torch.randint(0, dataset_size, size=(32,)),
    "target_sample_idx": torch.randint(0, dataset_size, size=(32,)),
}


def get_data(keys):
    return {k: example_data[k] for k in keys}

### [Adversarial Discriminative Domain Adaptation](https://arxiv.org/abs/1702.05464) (ADDA)

In [ ]:
from pytorch_adapt.hooks import ADDAHook

# make Target model
T = copy.deepcopy(G)
T_opt = torch.optim.Adam(T.parameters())
hook = ADDAHook(g_opts=[T_opt], d_opts=[D_opt])

models = {"G": G, "C": C, "D": D, "T": T}
data = get_data(["src_imgs", "target_imgs", "src_domain", "target_domain"])
losses, _ = hook({}, {**models, **data})
pprint(losses)

### [Larger Norm More Transferable: An Adaptive Feature Norm Approach for Unsupervised Domain Adaptation](https://arxiv.org/abs/1811.07456) (AFN)

In [ ]:
from pytorch_adapt.hooks import AFNHook, ClassifierHook

hook = ClassifierHook(opts=[G_opt, C_opt], post=[AFNHook()])

models = {"G": G, "C": C}
data = get_data(["src_imgs", "target_imgs", "src_labels"])
losses, _ = hook({}, {**models, **data})
pprint(losses)

### [Domain Adaptation with Auxiliary Target Domain-Oriented Classifier](https://arxiv.org/abs/2007.04171) (ATDOC)

In [ ]:
from pytorch_adapt.hooks import ATDOCHook, ClassifierHook

atdoc = ATDOCHook(dataset_size=10000, feature_dim=100, num_classes=10)
hook = ClassifierHook(opts=[G_opt, C_opt], post=[atdoc])

models = {"G": G, "C": C}
data = get_data(["src_imgs", "target_imgs", "src_labels", "target_sample_idx"])
losses, _ = hook({}, {**models, **data})
pprint(losses)

### [Towards Discriminability and Diversity: Batch Nuclear-norm Maximization under Label Insufficient Situations](https://arxiv.org/abs/2003.12237) (BNM)

In [ ]:
from pytorch_adapt.hooks import BNMHook, ClassifierHook

hook = ClassifierHook(opts=[G_opt, C_opt], post=[BNMHook()])

models = {"G": G, "C": C}
data = get_data(["src_imgs", "target_imgs", "src_labels"])
losses, _ = hook({}, {**models, **data})
pprint(losses)

### [Transferability vs. Discriminability: Batch Spectral Penalization for Adversarial Domain Adaptation](http://proceedings.mlr.press/v97/chen19i.html) (BSP)

In [ ]:
from pytorch_adapt.hooks import BSPHook, ClassifierHook
from pytorch_adapt.weighters import MeanWeighter

weighter = MeanWeighter(weights={"bsp_loss": 1e-3})
hook = ClassifierHook(opts=[G_opt, C_opt], post=[BSPHook()], weighter=weighter)

models = {"G": G, "C": C}
data = get_data(["src_imgs", "target_imgs", "src_labels"])
losses, _ = hook({}, {**models, **data})
pprint(losses)

### [Conditional Adversarial Domain Adaptation](https://arxiv.org/abs/1705.10667) (CDAN)

In [ ]:
from pytorch_adapt.hooks import CDANHook
from pytorch_adapt.layers import RandomizedDotProduct

feature_combiner = RandomizedDotProduct(in_dims=[100, 10], out_dim=100)
hook = CDANHook(g_opts=[G_opt, C_opt], d_opts=[D_opt])

models = {"G": G, "C": C, "D": D}
misc = {"feature_combiner": feature_combiner}
data = get_data(
    ["src_imgs", "target_imgs", "src_labels", "src_domain", "target_domain"]
)
losses, _ = hook({}, {**models, **misc, **data})
pprint(losses)

### [Deep CORAL: Correlation Alignment for Deep Domain Adaptation](https://arxiv.org/abs/1607.01719) (CORAL)

In [ ]:
from pytorch_adapt.hooks import AlignerPlusCHook
from pytorch_adapt.layers import CORALLoss

hook = AlignerPlusCHook(opts=[G_opt, C_opt], loss_fn=CORALLoss(), softmax=False)

models = {"G": G, "C": C}
data = get_data(["src_imgs", "target_imgs", "src_labels"])
losses, _ = hook({}, {**models, **data})
pprint(losses)

### [Domain-Adversarial Training of Neural Networks](https://arxiv.org/abs/1505.07818) (DANN)

In [ ]:
from pytorch_adapt.hooks import DANNHook

hook = DANNHook(opts=[G_opt, C_opt, D_opt])

models = {"G": G, "C": C, "D": D}
data = get_data(
    ["src_imgs", "target_imgs", "src_labels", "src_domain", "target_domain"]
)
losses, _ = hook({}, {**models, **data})
pprint(losses)

### [Simultaneous Deep Transfer Across Domains and Tasks](https://arxiv.org/abs/1510.02192) (Domain Confusion)

In [ ]:
from pytorch_adapt.hooks import DomainConfusionHook

# D has to output 2 values instead of the usual 1
D_ = torch.nn.Linear(100, 2)
D_opt_ = torch.optim.Adam(D_.parameters())

hook = DomainConfusionHook(g_opts=[G_opt, C_opt], d_opts=[D_opt_])

models = {"G": G, "C": C, "D": D_}
data = get_data(
    ["src_imgs", "target_imgs", "src_labels", "src_domain", "target_domain"]
)
losses, _ = hook({}, {**models, **data})
pprint(losses)

### GAN

In [ ]:
from pytorch_adapt.hooks import GANHook

hook = GANHook(g_opts=[G_opt, C_opt], d_opts=[D_opt])

models = {"G": G, "C": C, "D": D}
data = get_data(
    ["src_imgs", "target_imgs", "src_labels", "src_domain", "target_domain"]
)
losses, _ = hook({}, {**models, **data})
pprint(losses)

### [Gradually Vanishing Bridge for Adversarial Domain Adaptation](https://arxiv.org/abs/2003.13183) (GVB)

In [ ]:
from pytorch_adapt.hooks import GVBHook
from pytorch_adapt.layers import ModelWithBridge

# Discriminator comes after classifier,
# so the input shape is num_classes instead of feature size
D_ = torch.nn.Sequential(torch.nn.Linear(10, 1), torch.nn.Flatten(start_dim=0))

# Add bridges
C_ = ModelWithBridge(C)
D_ = ModelWithBridge(D_)
C_opt_ = torch.optim.Adam(C_.parameters())
D_opt_ = torch.optim.Adam(D_.parameters())

hook = GVBHook(opts=[G_opt, C_opt_, D_opt_])

models = {"G": G, "C": C_, "D": D_}
data = get_data(
    ["src_imgs", "target_imgs", "src_labels", "src_domain", "target_domain"]
)
losses, _ = hook({}, {**models, **data})
pprint(losses)

### Information Maximization (IM)

In [ ]:
from pytorch_adapt.hooks import ClassifierHook, TargetDiversityHook, TargetEntropyHook

hook = ClassifierHook(
    opts=[G_opt, C_opt], post=[TargetEntropyHook(), TargetDiversityHook()]
)

models = {"G": G, "C": C}
data = get_data(["src_imgs", "target_imgs", "src_labels"])
losses, _ = hook({}, {**models, **data})
pprint(losses)

### [Information-Theoretical Learning of Discriminative Clusters for Unsupervised Domain Adaptation](https://icml.cc/2012/papers/566.pdf) (ITL)

In [ ]:
from pytorch_adapt.hooks import (
    ClassifierHook,
    ISTLossHook,
    TargetDiversityHook,
    TargetEntropyHook,
)

hook = ClassifierHook(
    opts=[G_opt, C_opt],
    post=[ISTLossHook(), TargetEntropyHook(), TargetDiversityHook()],
)

models = {"G": G, "C": C}
data = get_data(
    ["src_imgs", "target_imgs", "src_labels", "src_domain", "target_domain"]
)
losses, _ = hook({}, {**models, **data})
pprint(losses)

### [Deep Transfer Learning with Joint Adaptation Networks](https://arxiv.org/abs/1605.06636) (JMMD)

In [ ]:
from pytorch_adapt.hooks import AlignerPlusCHook, JointAlignerHook
from pytorch_adapt.layers import MMDLoss
from pytorch_adapt.layers.utils import get_kernel_scales

kernel_scales = get_kernel_scales(low=-3, high=3, num_kernels=10)
loss_fn = MMDLoss(kernel_scales=kernel_scales)
aligner_hook = JointAlignerHook(loss_fn=loss_fn)
hook = AlignerPlusCHook(opts=[G_opt, C_opt], aligner_hook=aligner_hook)

models = {"G": G, "C": C}
data = get_data(["src_imgs", "target_imgs", "src_labels"])
losses, _ = hook({}, {**models, **data})
pprint(losses)